In [53]:
import requests
import json
import os
from bs4 import BeautifulSoup
from datetime import datetime
import pprint

In [84]:
source = 'canadafreepress'
date = '02-28'

filepath = '/mnt/c/Users/Brennan/Dropbox/MIDS/w210-capstone/data/notCredible/{0}/{1}.html'.format(source,date)

with open(filepath) as f:
    soup = BeautifulSoup(f, 'lxml')

In [85]:
# Pull out article links
links = []
i=0

for div in soup.findAll('div', attrs={'class':'container home'}):
    for article in div.findAll('article'):
        for h in article.findAll('h3'):
            for link in h.findAll('a'):
                links.append(link.get('href'))
                  
links = set(links)
links = list(links)

In [86]:
i=0
for link in links:
    r = requests.get(link)
    soup = BeautifulSoup(r.text, 'lxml')
    title = soup.title.string
    pubdate = soup.find('div', attrs={"id":"NmWg3225"}).find('em').text
    pubdate = str(datetime.strptime(pubdate, "%B %d, %Y"))
    if date in pubdate:
        date_longform = '{0}-2017'.format(date)
        contents = []
        for div in soup.find('div', attrs={"id":"NmWg3225"}):
            for p in div.findAll('p'):
                contents.append(p)
                
        article = {}
        content = ''.join(str(p) for p in contents)
        article["title"] = title
        article["url"] = link
        article["date"] = date_longform
        article["source"] = source
        article["content"] = content
    
        outfile = '/mnt/c/Users/Brennan/Dropbox/MIDS/w210-capstone/data/notCredible/{0}/articles/{2}/article{1}_{2}.txt'.format(source,i,date)
        os.makedirs(os.path.dirname(outfile), exist_ok=True)
        i+=1

        with open(outfile, 'w') as f:
            json.dump(article, f)